In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [40]:
df=pd.read_csv("Fraud_check.csv")

In [41]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 4E28-3FB0

 Directory of C:\Users\HP\Desktop\Assingments\assingment no.15(Random Forest)

12/16/2022  04:40 PM    <DIR>          .
12/16/2022  04:40 PM    <DIR>          ..
12/16/2022  03:40 PM    <DIR>          .ipynb_checkpoints
12/16/2022  03:11 PM            17,023 Company_Data.csv
12/16/2022  03:40 PM            47,282 Company_Data_Decision_Tree.ipynb
12/15/2022  04:55 PM         1,297,756 CompanyData_RF.ipynb
12/16/2022  03:31 PM            18,826 Fraud_check.csv
12/16/2022  04:40 PM            47,889 Fraud_check_RF.ipynb
12/16/2022  03:13 PM             1,378 problem statement(company data).txt
12/16/2022  03:11 PM               133 Problem_statement(Fraudcheck).txt
               7 File(s)      1,430,287 bytes
               3 Dir(s)  139,439,779,840 bytes free


In [42]:
df

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [43]:
df.isna().sum()

Undergrad          0
Marital_Status     0
Taxable_Income     0
City_Population    0
Work_Experience    0
Urban              0
dtype: int64

In [44]:
df[df['Taxable_Income']<= 30000]

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
10,NO,Single,29732,102602,19,YES
12,NO,Divorced,11794,148033,14,YES
16,YES,Divorced,24987,34551,29,NO
19,NO,Single,10987,126953,30,YES
21,YES,Divorced,14310,29106,7,YES
...,...,...,...,...,...,...
544,NO,Single,29916,133877,21,YES
557,YES,Married,12810,157421,10,NO
567,YES,Single,16316,129912,30,NO
575,NO,Divorced,10735,95975,13,YES


In [45]:
#Creating a new column called Fraud_Check 
group_names = ['Risky', 'Good']
df['Fraud_Check']=pd.cut(df['Taxable_Income'], bins = [0,30000,1000000], labels = group_names)

In [46]:
for i in range(0,len(df['Taxable_Income'])):
    if df['Taxable_Income'][i]<=30000:
        df['Fraud_Check'][i] = 'Risky'
    else:
        df['Fraud_Check'][i] = 'Good'

In [47]:
df.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,Fraud_Check
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good


In [48]:
df["Taxable_Income"].describe()

count      600.000000
mean     55208.375000
std      26204.827597
min      10003.000000
25%      32871.500000
50%      55074.500000
75%      78611.750000
max      99619.000000
Name: Taxable_Income, dtype: float64

In [49]:
df['Fraud_Check'].value_counts()

Good     476
Risky    124
Name: Fraud_Check, dtype: int64

In [50]:
df["Fraud_Check"]=df["Fraud_Check"].replace(['Risky','Good'],[0,1])
df['Urban']=df['Urban'].replace(['YES','NO'],[1,0])
df['Undergrad']=df['Undergrad'].replace(['YES','NO'],[1,0])
df["Marital_Status"]=df["Marital_Status"].replace(['Single','Divorced','Married'],[0,1,2])

In [51]:
df

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,Fraud_Check
0,0,0,68833,50047,10,1,1
1,1,1,33700,134075,18,1,1
2,0,2,36925,160205,30,1,1
3,1,0,50190,193264,15,1,1
4,0,2,81002,27533,28,0,1
...,...,...,...,...,...,...,...
595,1,1,76340,39492,7,1,1
596,1,1,69967,55369,2,1,1
597,0,1,47334,154058,0,1,1
598,1,2,98592,180083,17,0,1


In [52]:
df['Fraud_Check'].value_counts()

1    476
0    124
Name: Fraud_Check, dtype: int64

In [53]:
df['Urban'].value_counts()

1    302
0    298
Name: Urban, dtype: int64

In [54]:
df['Marital_Status'].value_counts()

0    217
2    194
1    189
Name: Marital_Status, dtype: int64

# Splitting data in x and y variable

In [55]:
df.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban,Fraud_Check
0,0,0,68833,50047,10,1,1
1,1,1,33700,134075,18,1,1
2,0,2,36925,160205,30,1,1
3,1,0,50190,193264,15,1,1
4,0,2,81002,27533,28,0,1


In [56]:
df.columns

Index(['Undergrad', 'Marital_Status', 'Taxable_Income', 'City_Population',
       'Work_Experience', 'Urban', 'Fraud_Check'],
      dtype='object')

In [57]:
df_new=df[['Undergrad', 'Marital_Status', 'City_Population',
       'Work_Experience', 'Urban','Fraud_Check']]

In [58]:
df_new.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,Fraud_Check
0,0,0,50047,10,1,1
1,1,1,134075,18,1,1
2,0,2,160205,30,1,1
3,1,0,193264,15,1,1
4,0,2,27533,28,0,1


In [59]:
X=df_new.iloc[:,:-1]
Y=df_new.iloc[:,-1]

In [60]:
X.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban
0,0,0,50047,10,1
1,1,1,134075,18,1
2,0,2,160205,30,1
3,1,0,193264,15,1
4,0,2,27533,28,0


In [61]:
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: Fraud_Check, dtype: int64

In [62]:
df_new.columns

Index(['Undergrad', 'Marital_Status', 'City_Population', 'Work_Experience',
       'Urban', 'Fraud_Check'],
      dtype='object')

# Splitting data into train and test data

In [63]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=10)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((480, 5), (120, 5), (480,), (120,))

In [64]:
x_train

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban
284,0,1,188383,22,1
423,0,2,80249,19,1
525,1,0,75307,23,0
519,1,0,162102,25,1
12,0,1,148033,14,1
...,...,...,...,...,...
369,0,2,89122,28,0
320,1,0,78897,11,1
527,0,2,180274,27,0
125,1,1,155669,11,0


In [65]:
y_train

284    0
423    1
525    1
519    1
12     0
      ..
369    1
320    1
527    1
125    1
265    0
Name: Fraud_Check, Length: 480, dtype: int64

# Doing Grid Search CV To find Best Hyperparameter

In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
model=RandomForestClassifier()
parameters={
    
    "n_estimators":[130, 140, 150, 160],
    "max_features":[2,3,4,5]
}

In [68]:
grid_DT=GridSearchCV(model,parameters,cv=10)

In [69]:
grid_DT.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_features': [2, 3, 4, 5],
                         'n_estimators': [130, 140, 150, 160]})

In [72]:
grid_DT.best_score_

0.7645833333333334

In [75]:
grid_DT.best_params_

{'max_features': 2, 'n_estimators': 130}

In [85]:
final_model=RandomForestClassifier(max_features=2, n_estimators=130)

In [87]:
final_model.fit(x_train,y_train)

RandomForestClassifier(max_features=2, n_estimators=130)

In [89]:
print(classification_report(y_test,final_model.predict(x_test)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.76      0.88      0.82        94

    accuracy                           0.69       120
   macro avg       0.38      0.44      0.41       120
weighted avg       0.60      0.69      0.64       120



In [90]:
final_model.feature_importances_

array([0.03725401, 0.0659378 , 0.53408095, 0.32783498, 0.03489226])